 <h1><center>Pam Search test</center></h1>



<div style="text-align: right">Lorenzo federico Signorini, 23/09/2019</div>

<div style="text-align: right">Università degli studi di Trento</div>


23/09/2019
Let' s try to  extract spacer sequences and lign them to all known contigs for the following Cas9s:
>8774__OBKKDPJE_00146__BritoIL_2016__M2.46.ST__bin.47;length1073;#sequences1
>8774__IBLHLBHI_00561__BritoIL_2016__W2.19.ST__bin.40;length1073;#sequences1

>8774__LLPOPKBG_01052__QinN_2014__HD-66__bin.16;length1073;#sequences1

>15299__PAOEJHHN_00966__FengQ_2015__SID531128__bin.48;15299__MOACNHFO_01282__XieH_2016__YSZC12003_36674__bin.32;length1074;#sequences2
>15299__LPEAHEHG_01071__ChengpingW_2017__AS43raw__bin.51;length1074;#sequences1

>4329__CJFLGLGO_01367__Obregon-TitoAJ_2015__SM40__bin.4;length1079;#sequences1

>4329__GNIJMFKL_00179__Obregon-TitoAJ_2015__HCO64__bin.33;length1079;#sequences1

>4329__OOELONHJ_00933__Obregon-TitoAJ_2015__SM01__bin.19;length1079;#sequences1

Each one of these is actually the centroid of a cluster of sequences, moreover, some of these actually are identical sequences from different genomes. It would be nice to see whether their CRISPR arrays are similar, too.


In [115]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import subprocess
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# for clustal alignments
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
from Bio.Align import AlignInfo
# to make logos
import logomaker as lm

sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
import originalpath
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus


feature="Cas9" #WARNING!!! CHANGE THIS!!
datadir="/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/8pamsearch/"


In [116]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy', 'prokka_cas9'],
      dtype='object')

da runnare tramite python script 

In [117]:
seqid ="LEAPHKCP_01380"   # MANUAL INPUT

dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Sample Name"].iloc[0]
#
full_genome_full_path, annotation_folder=originalpath.print_path(genomename)
#
old_genomename, old_dataset=filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset)
old_path="/shares/CIBIO-Storage/CM/scratch/users/e.pasolli/projects/binning/genomes_comp50_cont05/"+old_dataset+"/"

#
print("Sequence ID: ", seqid)
print("Contig:", contigname)
print("Dataset:",dataset)
print("Genome name", genomename)
print("Sample name:", samplename)
print("SGB ID:", SGB)
print("Cas9 length:", len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print("----------------------------------------------------------------------")
print("old epasolli genomename:", old_genomename, old_dataset )

print("TESTING, CHANGE DATADIR IF RUNNING PER DAVVERO")
outdir=datadir+"out/test/"#+seqid+"__"+genomename+"/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

Sequence ID:  LEAPHKCP_01380
Contig: NODE_1601_length_16622_cov_6.41836
Dataset: BritoIL_2016
Genome name BritoIL_2016__M2.36.ST__bin.48
Sample name: M2.36.ST
SGB ID: 9311
Cas9 length: 1065
----------------------------------------------------------------------
old epasolli genomename: BritoIL_2016__M2.36.ST__bin.48 BritoIL_2016
TESTING, CHANGE DATADIR IF RUNNING PER DAVVERO


In [118]:
# 1 get CRISPR spacer sequence
# get CRISPR spacer and repeat sequence
cr=locus.CRISPRarray(feature=feature, contigname=contigname, genomename=genomename, datasetname=dataset)
cr.get_CRISPR_array()
spacers=cr.spacers
#repeats=cr.repeats
#repeat_start_pos=cr.repstartpos
#unique_repeats=np.unique(repeats)
#unique_spacers=np.unique(spacers)
#todo I think we only care about spacers, no repeats, so all lines above commented out can be cancelled
print("Spacers", spacers )


Spacers ['TTGTTTTGGCGGCGTTGAACGCGTCGGCGA', 'ATTAAGTGGGTTGATTATTACTGATATTTG', 'ATACGGTTGAAGTCATTGCCGCAAGTTCAG', 'CTTTGATTTTCATTTGTTTTACTCCTTTTC', 'AACATGGACGGTTACCTGTTGAAGCGTATC', 'CGGGGTCCCTTACGGGGCCCCTTGGTGTAG', 'GCTTTCAAAGACGCTATTTCCTTGTCTTTT', 'CGCGGAAACGGGCGTATTCGGAAAGGTATC', 'TGTAAATCAAATCGTACGCACGAAACGGAA', 'CTTTCGCAGTTTCGTTATCCACTAAAGCTA', 'TGGCGGGCTTGGGTCTGATTTTGTGGTATA', 'AAACGGGCGCGCAGTATGATGCGGTTGTG', 'CCCGCGACAATCGCACCGCCTGCGTTCGCG', 'AGAACAAACATATTATCCACGCAAGAAAGGA', 'AATGGCGCTCGACTTTGTTTCGAAGTTTAG', 'TCGTTAGAGTTGCGCGGGAGGGTATCAACG', 'TTTTTATAAGTTAGCTTTACCCTGCTAAAG', 'CGACGCGCCCTTGGCATCGATTGAAAGACG', 'CTGTACACGAACGGCGACGGTGCGGGCTTC', 'GTCCTTGCCGTGCTTATAAACACAATAGTC', 'TCCGCAGTCCTGCGGTTGTTTCGGCGACCG']


In [119]:
#2. build temporary query file for blastn search, with spacers
blast_folder=outdir
os.chdir(blast_folder)
tempfile=open(outdir+"temp_spacer_seq", "w")
tempfile.close()
tempfile=open(outdir+"temp_spacer_seq", "a")
for n, spacer in enumerate(spacers):
    tempfile.write(">spacer"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+spacer+"\n") 
tempfile.close()

### BLAST. Must parallelize across ALL samples in the world

In [104]:
# fare un log, per ogni query (contig di partenza), che printi quell'output

In [105]:
# runan la cell qua sotto, e capisci la seq downstream come indicizzarla x averne 50

In [100]:
#3. Blast query file against db

# BackhedF_2015__SID130_M.contigs_filtered.fasta,    questo nel contig  NODE_16046_length_1718_cov_1.75827 non funge la seq upstream. esempio
# mentre BackhedF_2015__SID26_12M_megahit.contigs_filtered.fasta ha la seq DOWNStream corta (come accate ad altre), nel contig NODE_6634_length_2333_cov_7.0000_ID_14245
# altra downstream corta, x fare un dataset diverso: LawrenceA_2015__LD-Run2-11_megahit.contigs_filtered     NODE_1465_length_4048_cov_9.0000_ID_34458
print("Running blastn of query file agianst all contigs")
target_sample_file="VogtmannE_2016__MMRS68910755ST-27-0-0.contigs_filtered.fasta"#old_dataset+"__"+old_genomename.split("__")[1] #
dbfile=datadir+"VogtmannE_2016/"+target_sample_file # dataset+"/"+target_sample_file+".contigs_filtered.fasta" #
blastoutfile=outdir+seqid+"_vs_"+target_sample_file+".blastout"
blastn_command = "/home/lorenzo.signorini/ncbi-blast-2.10.0+/bin/blastn -out "+blastoutfile+\
    " -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send\
    evalue sstrand\" -query temp_spacer_seq -db "+dbfile+" -evalue 0.001 -word_size 12"
print(blastn_command)
subprocess.Popen(blastn_command, shell=True)

#per dopo pranzo, fai uno script che fa una lista di nomi di tti icontigs filtered,riguarda se i contigs_filtered siano solamente i non assemblati (ma non credo),e se c'è mdo nel file vecchio di vedere se le hit che avevi venivano da contigs non allineati. la risposta è: lo facevi con chewck if unbinned

Running blastn of query file agianst all contigs
/home/lorenzo.signorini/ncbi-blast-2.10.0+/bin/blastn -out /shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/8pamsearch/out/test/LEAPHKCP_01380_vs_VogtmannE_2016__MMRS68910755ST-27-0-0.contigs_filtered.fasta.blastout -outfmt "6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send    evalue sstrand" -query temp_spacer_seq -db /shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/8pamsearch/VogtmannE_2016/VogtmannE_2016__MMRS68910755ST-27-0-0.contigs_filtered.fasta -evalue 0.001 -word_size 12


In [122]:
# 4 blast output parsing
# 4.1 Print all output
protospacers_of={} # save all putative protospacers, for every contig
f=open(blastoutfile)
lines=f.readlines()
print("query seq\tMatched_contig\t% identity\tquery length\talignment length\t# mismatches\t# gap opens \t query sequence\taligned part of subject sequence\tstart in subject\tend in subject\texpect. value\tmatched contig strand\n----------------------------------------------------------------------------------------------------------------------")
for line in lines:
    target_cont=line.strip("\n").split()[1]  # contig name of target (putative protospacer)

    if target_cont!=contigname: #filter out putative protospacers coming from same contig
        print(line)
        if not target_cont in protospacers_of.keys():
            protospacers_of[target_cont]=[]
        # add putative protospacer sequence to dictionary
        protospacers_of[target_cont].append((int(line.strip("\n").split()[-4]),int(line.strip("\n").split()[-3]),line.strip("\n").split()[-5]))

query seq	Matched_contig	% identity	query length	alignment length	# mismatches	# gap opens 	 query sequence	aligned part of subject sequence	start in subject	end in subject	expect. value	matched contig strand
----------------------------------------------------------------------------------------------------------------------
spacer4|NODE_1601_length_16622_cov_6.41836|BritoIL_2016__M2.36.ST__bin.48|LEAPHKCP_01380	NODE_16721_length_1719_cov_2.81731	96.296	30	27	0	1	TGATTTTCATTTGTTTTACTCCTTTTC	TGATTTTCATTTGTTTT-CTCCTTTTC	1092	1067	1.18e-04	minus



In [128]:
#voglio 6 nt a valle (downstream) di protospacer AAAAAAAAAX: ne ho solo 5 (BBBBB)
# to understand how string slicing works and hos slicing is different than indexing with strings
#and it behaves differently than lists
l="ccccCYAAAAAAAAAXBbbbbZ"
lrevcomp=Seq(l).reverse_complement()
print(lrevcomp)
start=6 #output da blast
end=16 # output da blast
#lo voglio lungo 50
print("full sequence",l)
print("extract sequence using blast start and end indexes")
print("my protospacer: l[start-1:end]", l[start-1:end])
print("last letter of protospacer: l[end-1]]", l[end-1])
print("First letter of downstream flanking region: l[end]",l[end])
print("ALL downstream sequence l[end:]",l[end:])
print("iI know..")
print("first (or last) letter of upstream flanking: l[start]", l[start])
print("ALL upstream sequence l[:start-1]",l[:start-1])


print("I know..")
last_pos=len(l)
flanking_length=6
print(start,flanking_length)
if start>=(flanking_length+1):
    upstream_start=start-(flanking_length+1) 
    print(upstream_start)
    added_nucleotides=""
else:
    upstream_start=0
    added_nucleotides="X"*((flanking_length+1)-start)
upseq=added_nucleotides+l[upstream_start:start-1] #immedia
print(upseq,len(upseq))
    

ZvvvvVXTTTTTTTTTRGgggg
full sequence ccccCYAAAAAAAAAXBbbbbZ
extract sequence using blast start and end indexes
my protospacer: l[start-1:end] YAAAAAAAAAX
last letter of protospacer: l[end-1]] X
First letter of downstream flanking region: l[end] B
ALL downstream sequence l[end:] BbbbbZ
iI know..
first (or last) letter of upstream flanking: l[start] A
ALL upstream sequence l[:start-1] ccccC
I know..
6 6
XccccC 6


In [130]:
### 2 putative viral contigs extraction
#TODO note to self: THERE IS ONLY ONE contigname now, but se ne faccio
# di piu, will need to map target contigs to query contig in another dictionary
# extract thje +50, -50 sequence flanking protospacer, for every contig (ASSUMING 1 contig = 1 protospacer) TODOable
# for the particular locus studied, for every contig of putative protospacers
print("Extracting flanking sequences. There are", len(list(protospacers_of.keys())), "contigs with\
putative protospacers.")
flanking_sequences_of_putative_protospacers_of={}
f=open(outdir+"flanking_sequences_of_putative_protospacers_"+target_sample_file,"w")
f=open(outdir+"flanking_sequences_of_putative_protospacers_"+target_sample_file,"a")
for target_contig in protospacers_of.keys():
    print("Searching for flanking sequences of protospacer from contig:\n",target_contig,"\n")

    # check if binned
    print("checking if binned")
    target_bin='0'
    os.chdir(old_path)
    for file in os.listdir():
        if file.startswith(target_sample_file):
            for record in SeqIO.parse(file, "fasta"):
                if record.id==target_contig:
                    target_bin=file.rstrip(".fa")
    #
    
    for record in SeqIO.parse(dbfile, "fasta"):
        if record.id==target_contig:
            #contig_sequence_with_putative_protospacer[contigname]=record.seq
            for start, end, matched_seq in protospacers_of[target_contig]:
                sequence=record.seq
                revcomp=False
                if start>end:
                    print("Match on minus, taking reverse complement")
                    s=start
                    start=end
                    end=s
                    revcomp=True
                extracted_protospacer=sequence[start-1:end].reverse_complement() if revcomp else sequence[start-1:end]
          #      if not extracted_protospacer==matched_seq:
           #         print("matched seq:", matched_seq)
            #        print("extracted:", extracted_protospacer)
             #       raise Exception("Extracted sequence differs:\nExtracted sequence reverse complement?"+str(revcomp))
                print("Is extracted putative protospacer equal to Blasted?",extracted_protospacer==matched_seq)
                print("original match:",matched_seq)
                print("start:",start,"end:",end)
                print("protospacer extracted:",extracted_protospacer)
                flanking_length=50
                print("upstream sequence")
                # Warning! Indexing on strings is different than slicing on strings
                if start>=(flanking_length+1):
                    upstream_start=start-(flanking_length+1)
                    added_nucleotides=""
                else:
                    upstream_start=0
                    added_nucleotides="X"*((flanking_length+1)-start)
                print("TEST: should end with TACC")
                upseq=added_nucleotides+sequence[upstream_start:start-1] #immediately flanking, no overlap
                print("TEST: should end with TACC")
                
                print("downstream sequence")
                last_pos=len(sequence)
                if last_pos-end>=flanking_length:
                    downstream_end=end+flanking_length
                    added_nucleotides=""
                else: #check it works. probly no need
                    downstream_end=last_pos
                    added_nucleotides="X"*(flanking_length-(last_pos-end))
                downseq=sequence[end:downstream_end]+added_nucleotides  #immediately flanking, no overlap
                
                # pezzo revcomp
                if revcomp:
                    temp=downseq
                    downseq=upseq.reverse_complement()
                    upseq=temp.reverse_complement()

                print("up",upseq,len(upseq))
                print("down:",downseq,len(downseq))
                f.write(target_contig+","+str(upseq)+","+str(downseq)+","+target_sample_file+","+target_bin+"\n")
                #todo add if is binned to the thing above
                #todo: remove these below
                if not target_contig in flanking_sequences_of_putative_protospacers_of.keys():
                    flanking_sequences_of_putative_protospacers_of[target_contig]=[]
                flanking_sequences_of_putative_protospacers_of[target_contig].append((upseq,downseq))
                
            print(flanking_sequences_of_putative_protospacers_of)

            break
f.close()

Extracting flanking sequences. There are 1 contigs withputative protospacers.
Searching for flanking sequences of protospacer from contig:
 NODE_16721_length_1719_cov_2.81731 

checking if binned
Match on minus, taking reverse complement
Is extracted putative protospacer equal to Blasted? False
original match: TGATTTTCATTTGTTTT-CTCCTTTTC
start: 1067 end: 1092
protospacer extracted: TGATTTTCATTTGTTTTCTCCTTTTC
upstream sequence
TEST: should end with TACC
TEST: should end with TACC
downstream sequence
up TAAACTTACTCCCATTGTTACTGCCAGCATGCCTCCCACTAATTTGTTTC 50
down: ATTTTTTCATGCGGACGCATCCTTCCAAAGCTCTTCCTTTACTGCCGAAA 50
{'NODE_16721_length_1719_cov_2.81731': [(Seq('TAAACTTACTCCCATTGTTACTGCCAGCATGCCTCCCACTAATTTGTTTC', SingleLetterAlphabet()), Seq('ATTTTTTCATGCGGACGCATCCTTCCAAAGCTCTTCCTTTACTGCCGAAA', SingleLetterAlphabet()))]}
